# Check Broken Links

In [1]:
import os
import pandas as pd
from glob import glob
from bs4 import BeautifulSoup

In [2]:
html_root = '../docs/'

In [3]:
# html_files = [f for f in os.listdir(html_root) if '.html' in f]
html_files = sorted([y for x in os.walk(html_root) for y in glob(os.path.join(x[0], '*.html'))])

In [4]:
df = pd.DataFrame()

for h in html_files:

    with open(h, 'r') as f:
        html_text = f.read()

    df_file = pd.DataFrame()
    soup = BeautifulSoup(html_text, features='html.parser')
    links = [link.get('href') for link in soup.find_all('a')]
    
    df_file['destination'] = links
    df_file['source'] = h
    
    df = pd.concat([df, df_file], ignore_index=True)

In [6]:
df['is_local'] = True
df.loc[df.destination.str.contains('http'), 'is_local'] = False

In [9]:
df['exists'] = df['destination'].apply(lambda x: os.path.exists(html_root + x))

In [7]:
links_to_check = df[df.is_local].groupby('destination').size()
# links_to_check

In [8]:
df['is_broken'] = (df.is_local) & ~(df.exists)

In [9]:
df.is_broken.sum()

758

In [10]:
df[df.is_broken].source.unique().tolist()

['counts.html', 'list.html']

In [10]:
df

,destination,source,is_local,exists
0,about.html,../docs/index.html,True,True
1,https://en.wikipedia.org/wiki/Advisory_Neighbo...,../docs/about.html,False,False
2,https://anc.dc.gov/,../docs/about.html,False,False
3,https://opendata.dc.gov/datasets/single-member...,../docs/about.html,False,False
4,http://www.thegreatermarin.org/,../docs/about.html,False,False
...,...,...,...,...
30748,../../people/people_list.html,../docs/people/Earl_E_Biglow.html,True,False
30749,../../counts.html,../docs/people/Earl_E_Biglow.html,True,False
30750,../../about.html,../docs/people/Earl_E_Biglow.html,True,False
30751,https://twitter.com/OpenANC,../docs/people/Earl_E_Biglow.html,False,False


In [13]:
os.path.exists(html_root + 'index.html')

True